In [2]:
# from nba_api.stats.static import players
#
#
# players.find_players_by_full_name("Lebron James")
#
# players.find_player_by_id()

TypeError: find_player_by_id() missing 1 required positional argument: 'player_id'

In [52]:
from nba_api.stats.endpoints import commonallplayers

from_year = 2010

total_players = commonallplayers.CommonAllPlayers().get_data_frames()[0]
total_players['TO_YEAR'] = total_players['TO_YEAR'].astype('int')
total_players = total_players[total_players['TO_YEAR'] > from_year]

#Ho controllato che in total_players, il PERSON_ID sia univoco
total_players

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
9,203518,"Abrines, Alex",Alex Abrines,0,2016,2018,alex_abrines,alex_abrines,0,,,,,None,Y,00
10,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2021,precious_achiuwa,precious_achiuwa,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
15,203112,"Acy, Quincy",Quincy Acy,0,2012,2018,quincy_acy,quincy_acy,0,,,,,None,Y,01
19,1629121,"Adams, Jaylen",Jaylen Adams,0,2018,2020,jaylen_adams,jaylen_adams,0,,,,,None,Y,01
20,203919,"Adams, Jordan",Jordan Adams,0,2014,2015,jordan_adams,jordan_adams,0,,,,,None,Y,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4713,203092,"Zeller, Tyler",Tyler Zeller,0,2012,2019,tyler_zeller,tyler_zeller,0,,,,,None,Y,00
4719,1627757,"Zimmerman, Stephen",Stephen Zimmerman,0,2016,2016,stephen_zimmerman,stephen_zimmerman,0,,,,,None,Y,01
4720,1627835,"Zipser, Paul",Paul Zipser,0,2016,2017,paul_zipser,paul_zipser,0,,,,,None,Y,01
4721,1627790,"Zizic, Ante",Ante Zizic,0,2017,2019,ante_zizic,ante_zizic,0,,,,,None,Y,01


In [53]:
from nba_api.stats.endpoints import playercareerstats
import pandas as pd
from tqdm import tqdm
import pickle

total_names = total_players[["PERSON_ID", "DISPLAY_FIRST_LAST"]]
total_stats = pd.DataFrame()


for index, row in tqdm(total_names.iterrows()):
    stats = playercareerstats.PlayerCareerStats(player_id=row["PERSON_ID"]).get_data_frames()[0]
    #stats = stats[stats[]] PRENDERE SOLO I DATI SUPERIORE ALL'ANNO FROM_YEAR
    #name = row[["DISPLAY_FIRST_LAST"]].repeat(stats.shape[0])
    #tot = pd.concat([stats, name], axis=1, ignore_index=True)
    total_stats = pd.concat([total_stats, stats],axis=0,ignore_index=True)

with open("player_stats.pkl",'wb') as f:
    pickle.dump(total_stats,f)

#Find lebron james totals stats
#playercareerstats.PlayerCareerStats(player_id=2544).get_data_frames()[0]

1471it [44:00,  1.79s/it]


#How it's structured this dataset
- Player_id --> id del giocatore(da ritrovare il nome con players.find_player_by_id())
- Season_id --> id della stagione
- ...other...
- GP --> Games Played
- GS --> Games ... (idk but similar to GP) (I think is not usefull)
- MIN Minutes Played
- FGM Field Goals Made --> Numero canestri fatti
- FGA Field Goals Attempted --> Numero tiri provati
- FG% Field Goal Percentage --> FGM/FGA, rapporto precisione tiri
- 3PM 3 Point Field Goals Made --> Numero canestri da 3 fatti
- 3PA 3 Point Field Goals Attempted --> Numero tiri da 3 provati
- 3P% 3 Point Field Goals Percentage --> 3PM/3PA, precisione tiri da 3
- FTM Free Throws Made --> Tiri liberi fatti
- FTA Free Throws Attempted --> tiri liberi provati
- FT% Free Throw Percentage --> FTM/FTA
- OREB Offensive Rebounds --> Rimbalzi conquistati in attacco
- DREB Defensive Rebounds --> rimbalzi conquistati in difesa
- REB Rebounds --> OREB + DREB, rimbalzi totali
- AST Assists
- TOV Turnovers --> palla persa
- STL Steals --> palla rubata
- BLK Blocks --> blocchi
- PF Personal Fouls --> falli personali
- DD2 Double Doubles --> doppie doppie
- TD3 Trible Doubles --> triple doppie
- PTS Points